In [1]:
import pandas as pd
import re

In [6]:
"aa aa223 aaaaaaa".replace(r"\b\w{1,3}\b", ' ')

'aa aa223 aaaaaaa'

In [8]:
#CAMBIAR AQUÍ EL ARCHIVO
jb = pd.read_csv("jb_part.csv")
jb["C"] = 1
dt = pd.read_csv("dt_part.csv")
jb["C"] = 0

In [9]:
df = jb.append(dt)

# CARACTERISTICAS DE LA BASE

In [18]:
#SELECCIÓN DE LA MUESTRA
df = df[(df["country"]=="United States of America") | (df["country"]=="United States")] #SOLO ESTADOS UNIDOS 
df = df[(df["continent"]=="North America")]

In [19]:
#¿NOS QUEDAMOS CON LOS NO MISSING VALUES DE CIUDAD?
# POR MI PARTE SI
# INCLUSO PODEMOS NO QUEDARNOS POR NO MISSING DE CIUDAD-ESTADO PARA HACER MÁS FÁCIL LA GRAFICACIÓN

In [20]:
df.shape

(39705, 23)

In [21]:
#VISUALIZACIÓN
df.head()

,Unnamed: 0,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,...,user_location,lat,long,city,country,continent,state,state_code,collected_at,C
1,38488,2020-10-17 13:32:59,1.317459e+18,Did the #democrats not try to impeach #Trump o...,2.0,2.0,Twitter Web App,4.779699e+07,Tom Dooley,tomofsnj,...,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,2020-10-21 05:39:00.255677220,0.0
10,739852,2020-11-08 01:07:09,1.325243e+18,@spectatorindex Lets first wait until all the ...,0.0,0.0,Twitter for iPhone,2.540326e+09,SupportIsrael,ISRAELsupportNL,...,"Boston, MA",42.360253,-71.058291,Boston,United States of America,North America,Massachusetts,MA,2020-11-09 18:39:37.699577,0.0
11,317561,2020-11-03 05:16:46,1.323494e+18,@JohnWDean This would make me sleep a lot bett...,1.0,0.0,Twitter Web App,8.742585e+08,Jonathan Greenberg,JournalistJG,...,"New York, NY",40.712728,-74.006015,New York,United States of America,North America,New York,NY,2020-11-04 09:47:28.623150,0.0
12,739853,2020-11-08 01:07:09,1.325243e+18,Of course this desperate man's certain that th...,158.0,76.0,Twitter for Android,1.016349e+18,Corren Love ☆BLACKLIVESMATTER☆,corren_love,...,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,2020-11-09 18:39:37.515584,0.0
19,231729,2020-10-29 15:02:23,1.321830e+18,I can’t wait to have a President and First Lad...,0.0,0.0,Twitter for iPhone,1.210572e+09,Juanito 🇲🇽🇺🇸,juannnchoo,...,"Atlanta, GA",33.749099,-84.390185,Atlanta,United States of America,North America,Georgia,GA,2020-10-30 15:00:34.607781,0.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39705 entries, 1 to 97091
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            39705 non-null  int64  
 1   created_at            39705 non-null  object 
 2   tweet_id              39705 non-null  float64
 3   tweet                 39705 non-null  object 
 4   likes                 39705 non-null  float64
 5   retweet_count         39705 non-null  float64
 6   source                39700 non-null  object 
 7   user_id               39705 non-null  float64
 8   user_name             39705 non-null  object 
 9   user_screen_name      39705 non-null  object 
 10  user_description      37430 non-null  object 
 11  user_join_date        39705 non-null  object 
 12  user_followers_count  39705 non-null  float64
 13  user_location         39705 non-null  object 
 14  lat                   39705 non-null  float64
 15  long               

In [23]:
df.describe()

,Unnamed: 0,tweet_id,likes,retweet_count,user_id,user_followers_count,lat,long,C
count,39705.000000,3.970500e+04,39705.000000,39705.000000,3.970500e+04,3.970500e+04,39705.000000,39705.000000,18302.0
mean,355277.449691,1.321777e+18,10.980960,2.584032,3.439326e+17,1.138893e+04,37.590496,-93.440561,0.0
std,254820.045511,2.697731e+15,291.829131,52.805664,5.053433e+17,1.023009e+05,4.894109,16.757791,0.0
min,2.000000,1.316529e+18,0.000000,0.000000,2.654000e+03,0.000000e+00,18.108266,-161.755833,0.0
25%,145630.000000,1.319459e+18,0.000000,0.000000,9.157615e+07,1.250000e+02,34.053691,-100.445882,0.0
50%,296445.000000,1.322245e+18,0.000000,0.000000,9.066751e+08,6.200000e+02,39.526121,-92.007126,0.0
75%,536473.000000,1.324154e+18,1.000000,0.000000,8.461837e+17,2.686000e+03,40.712728,-77.727883,0.0
max,970897.000000,1.325589e+18,35449.000000,6629.000000,1.325533e+18,3.264260e+06,65.538963,-65.883465,0.0


In [24]:
#NUMERO DE MISSING VALUES
df.isnull().sum()

Unnamed: 0                  0
created_at                  0
tweet_id                    0
tweet                       0
likes                       0
retweet_count               0
source                      5
user_id                     0
user_name                   0
user_screen_name            0
user_description         2275
user_join_date              0
user_followers_count        0
user_location               0
lat                         0
long                        0
city                    20232
country                     0
continent                   0
state                    6119
state_code               6119
collected_at                0
C                       21403
dtype: int64

In [25]:
#VALORES UNICOS DE COLUMNAS
for i in ["user_location", "city", "country", "continent", "state", "state_code"]:
    print(i, df[i].unique(), "\n")

user_location ['USA' 'Boston, MA' 'New York, NY' ... 'North Phoenix, Arizona, USA'
 'bimbo, nc' 'Nothern California, USA'] 

city [nan 'Boston' 'New York' 'Atlanta' 'Denver' 'Kansas City' 'Philadelphia'
 'Miami' 'Pittsburgh' 'Jacksonville' 'Houston' 'Portland' 'Washington'
 'Los Angeles' 'Chicago' 'San Francisco' 'Charleston' 'El Paso'
 'Minneapolis' 'Dallas' 'Scottsdale' 'San Jose' 'Sarasota' 'Boulder'
 'Sioux Falls' 'Nashville' 'San Antonio' 'Akron' 'Newark' 'Mobile'
 'Honolulu' 'Seattle' 'Columbus' 'Buffalo' 'Raleigh' 'Indianapolis'
 'Jersey City' 'Bakersfield' 'Phoenix' 'Stockton' 'Austin' 'Arlington'
 'Toms River' 'Memphis' 'Las Vegas' 'Tampa' 'Spokane' 'Fort Lauderdale'
 'North Port' 'Boise' 'Savannah' 'Madison' 'Bellevue' 'Rochester'
 'Iowa City' 'Oxnard' 'Santa Barbara' 'San Diego' 'Detroit' 'Yonkers'
 'Rancho Cucamonga' 'McKinney' 'Sacramento' 'Urbana' 'Cincinnati'
 'Baton Rouge' 'Long Beach' 'Orlando' 'Las Cruces' 'Baltimore' 'Milwaukee'
 'Saint Paul' 'Oakland' 'Palm Springs'

In [26]:
df.to_csv("final_db.csv")